In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd

query = "SELECT * FROM disclosure_events"
df = pd.read_sql(query, engine)
print(f"데이터베이스에서 조회된 공시 이벤트 수: {len(df)}")
df.head()


DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


데이터베이스에서 조회된 공시 이벤트 수: 31165


,id,stock_code,market,company_name,report_name,disclosed_at,summary_kr,raw,created_at,ret_1m,ret_3m,ret_10m,ret_15m,ret_30m,ret_60m,label,score
0,32519,KR7114190002,KOSDAQ,강원에너지,타법인주식및출자증권취득결정(자율공시),2022-07-01 01:03:17+00:00,"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...","2022.07.01 10:03:16\n기업명: 강원에너지(시가총액: 1,842억)\...",2025-08-09 19:13:15.234304,-0.14,0.00,-1.99,-3.55,-3.83,-3.69,neutral,0.9723
1,52054,KR7068270008,KOSPI,셀트리온,투자판단관련주요경영사항(CT-P42(아일리아 바이오시밀러) 임상 3상 결과 발표),2023-04-02 23:00:06+00:00,셀트리온은 당뇨병성 황반부종 환자를 대상으로 한 아일리아 바이오시밀러(CP-T42)...,2023.04.03 08:00:05\n기업명: 셀트리온(시가총액: 21조 948억)...,2025-08-09 19:13:27.639981,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2,33989,None,None,CBI,주요사항보고서(무상증자결정),2022-07-29 07:26:53+00:00,"CBI는 2022년 7월 29일 무상증자를 결정하였으며, 발행일은 9월 6일이고 기...","2022.07.29 16:26:52\n기업명: CBI(시가총액: 1,681억)\n보...",2025-08-09 19:13:15.234304,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
3,32520,KR7082920000,KOSDAQ,비츠로셀,단일판매ㆍ공급계약체결,2022-07-01 01:24:45+00:00,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,"2022.07.01 10:24:44\n기업명: 비츠로셀(시가총액: 2,477억)\n...",2025-08-09 19:13:15.234304,-0.85,0.43,0.00,1.28,0.85,0.85,positive,0.9959
4,64741,KR7381970003,KOSPI,케이카,영업(잠정)실적(공정공시),2023-11-01 23:57:28+00:00,"케이카는 2023년 3분기 매출액 5,286억, 영업이익 184억, 순이익 54억을...","2023.11.02 08:57:27\n기업명: 케이카(시가총액: 4,727억)\n보...",2025-07-08 17:09:01.716527,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [3]:
# market 종류와 각각의 비율 분석
market_counts = df['market'].value_counts()
print("시장별 공시 분포:")
print(market_counts)
print(f"\n총 공시 건수: {len(df)}")
print("\n시장별 비율:")
for market, count in market_counts.items():
    ratio = count / len(df) * 100
    print(f"{market}: {count}건 ({ratio:.1f}%)")


시장별 공시 분포:
market
KOSDAQ           13403
KOSPI            12392
KOSDAQ GLOBAL     1057
Name: count, dtype: int64

총 공시 건수: 31165

시장별 비율:
KOSDAQ: 13403건 (43.0%)
KOSPI: 12392건 (39.8%)
KOSDAQ GLOBAL: 1057건 (3.4%)


In [4]:
# disclosed_at을 datetime으로 변환 (이미 timezone-aware 상태)
df['disclosed_at'] = pd.to_datetime(df['disclosed_at'])

# 이미 timezone-aware이므로 tz_convert만 사용하여 KST로 변환
df['disclosed_at_kst'] = df['disclosed_at'].dt.tz_convert('Asia/Seoul')

# KST 시간에 따라 데이터프레임 정렬
df = df.sort_values('disclosed_at_kst')

# 시간만 추출 (KST 기준)
df['time_only'] = df['disclosed_at_kst'].dt.time



# 장중 시간 정의 (9:00 ~ 15:30)
import datetime
market_open = datetime.time(9, 0)
market_close = datetime.time(15, 30)

# 장중/장외 분류
df['market_session'] = df['time_only'].apply(
    lambda x: '장중' if market_open <= x <= market_close else '장외'
)

# 결과 집계
session_counts = df['market_session'].value_counts()
print("공시 발표 시간대별 분포:")
print(session_counts)
print(f"\n장중 비율: {session_counts['장중'] / len(df) * 100:.1f}%")
print(f"장외 비율: {session_counts['장외'] / len(df) * 100:.1f}%")


공시 발표 시간대별 분포:
market_session
장외    19745
장중    11420
Name: count, dtype: int64

장중 비율: 36.6%
장외 비율: 63.4%


In [5]:
import pandas as pd
from datetime import datetime

# 날짜 범위 설정
start_date = datetime(2022, 7, 1)
end_date = datetime(2023, 12, 28)

table = "disclosure_events"
query = f"""SELECT * FROM {table}
WHERE stock_code is not NULL
  AND summary_kr is not NULL
  AND market IN ('KOSPI', 'KOSDAQ', 'KOSDAQ GLOBAL') 
  AND (
      (EXTRACT(HOUR FROM disclosed_at AT TIME ZONE 'Asia/Seoul') BETWEEN 9 AND 14)
      OR
      (EXTRACT(HOUR FROM disclosed_at AT TIME ZONE 'Asia/Seoul') = 15
       AND EXTRACT(MINUTE FROM disclosed_at AT TIME ZONE 'Asia/Seoul') <= 30)
  )
  AND disclosed_at >= %(start_date)s
  AND disclosed_at <= %(end_date)s
ORDER BY id ASC"""

target_df = pd.read_sql(query, engine, params={"start_date": start_date, "end_date": end_date})

target_df

,id,stock_code,market,company_name,report_name,disclosed_at,summary_kr,raw,created_at,ret_1m,ret_3m,ret_10m,ret_15m,ret_30m,ret_60m,label,score
0,32519,KR7114190002,KOSDAQ,강원에너지,타법인주식및출자증권취득결정(자율공시),2022-07-01 01:03:17+00:00,"강원에너지(시가총액 1,842억 원)는 2022년 7월 1일 강원이솔루션의 주식을 ...","2022.07.01 10:03:16\n기업명: 강원에너지(시가총액: 1,842억)\...",2025-08-09 19:13:15.234304,-0.14,0.00,-1.99,-3.55,-3.83,-3.69,neutral,0.9723
1,32520,KR7082920000,KOSDAQ,비츠로셀,단일판매ㆍ공급계약체결,2022-07-01 01:24:45+00:00,비츠로셀이 2022년 6월 30일부터 2023년 10월 20일까지 방위사업청과 리튬...,"2022.07.01 10:24:44\n기업명: 비츠로셀(시가총액: 2,477억)\n...",2025-08-09 19:13:15.234304,-0.85,0.43,0.00,1.28,0.85,0.85,positive,0.9959
2,32521,KR7042370007,KOSDAQ,비츠로테크,단일판매ㆍ공급계약체결(자회사의 주요경영사항),2022-07-01 01:27:02+00:00,비츠로테크는 방위사업청과 리튬전지류 및 기뢰정비장비용 전지조립체 공급 계약을 체결했...,"2022.07.01 10:27:02\n기업명: 비츠로테크(시가총액: 1,821억)\...",2025-08-09 19:13:15.234304,0.00,-0.14,-0.29,0.14,0.72,1.01,positive,0.9999
3,32522,KR7014820005,KOSPI,동원시스템즈,신규시설투자등(자율공시),2022-07-01 01:43:15+00:00,동원시스템즈는 2022년 7월 1일부터 2023년 7월 31일까지 585억 원을 신...,"2022.07.01 10:43:14\n기업명: 동원시스템즈(시가총액: 1조 5,61...",2025-08-09 19:13:15.234304,0.00,0.18,0.37,0.55,0.74,0.00,neutral,0.6471
4,32524,KR7226950004,KOSDAQ,올릭스,투자판단관련주요경영사항(노인성 황반변성 환자에서 OLX10212의 안전성 및 내약성...,2022-07-01 02:30:09+00:00,올릭스는 노인성 황반변성 치료를 위한 OLX10212의 제1상 임상시험을 미국에서 ...,"2022.07.01 11:30:08\n기업명: 올릭스(시가총액: 2,946억)\n보...",2025-08-09 19:13:15.234304,-1.60,-1.37,-0.68,-1.37,-2.28,-2.51,positive,0.9998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9638,68075,KR7445090004,KOSDAQ,에이직랜드,주요사항보고서(회사합병결정),2023-12-27 05:31:32+00:00,에이직랜드는 반도체 디자인 서비스를 제공하는 자회사 탑에이직을 흡수합병하여 사업 통...,"2023.12.27 14:31:31\n기업명: 에이직랜드(시가총액: 6,396억)\...",2025-07-08 17:09:03.053194,0.33,0.17,-0.17,-0.67,-0.50,NaN,neutral,0.6333
9639,68076,KR7445090004,KOSDAQ,에이직랜드,주요사항보고서(회사합병결정),2023-12-27 05:33:57+00:00,에이직랜드는 반도체 디자인 서비스를 주력으로 하는 자회사 위즈마인드를 흡수합병하여 ...,"2023.12.27 14:33:56\n기업명: 에이직랜드(시가총액: 6,428억)\...",2025-07-08 17:09:03.053194,0.17,0.00,-0.17,-0.33,-0.50,NaN,positive,0.8485
9640,68077,KR7018310003,KOSDAQ,삼목에스폼,소송등의제기ㆍ신청(경영권분쟁소송),2023-12-27 05:48:48+00:00,삼목에스폼은 수원지방법원 평택지원에 장부 열람허용 가처분 신청 사건과 관련하여 영업...,"2023.12.27 14:48:47\n기업명: 삼목에스폼(시가총액: 2,436억)\...",2025-07-08 17:09:03.053194,-1.15,-0.12,-0.18,0.18,1.15,NaN,negative,0.9982
9641,68078,KR7012700001,KOSDAQ,리드코프,타법인주식및출자증권처분결정,2023-12-27 06:03:00+00:00,리드코프는 2024년 3월 31일까지 채권추심전문회사인 엘씨대부의 지분 11.71%...,"2023.12.27 15:02:59\n기업명: 리드코프(시가총액: 1,584억)\n...",2025-07-08 17:09:03.053194,0.00,-0.17,-0.17,0.00,NaN,NaN,neutral,0.9999


In [6]:
print("\n=== 기본 데이터 분석 ===")

# 컬럼 정보
print(f"컬럼 수: {len(target_df.columns)}")
print(f"컬럼명: {list(target_df.columns)}")

# 데이터 타입
print("\n데이터 타입:")
print(target_df.dtypes)

# 결측값 확인
print("\n결측값 현황:")
print(target_df.isnull().sum())

# 수익률 컬럼 확인
ret_columns = ["ret_1m", "ret_3m", "ret_10m", "ret_60m"]
print(f"\n수익률 컬럼: {ret_columns}")


=== 기본 데이터 분석 ===
컬럼 수: 17
컬럼명: ['id', 'stock_code', 'market', 'company_name', 'report_name', 'disclosed_at', 'summary_kr', 'raw', 'created_at', 'ret_1m', 'ret_3m', 'ret_10m', 'ret_15m', 'ret_30m', 'ret_60m', 'label', 'score']

데이터 타입:
id                            int64
stock_code                   object
market                       object
company_name                 object
report_name                  object
disclosed_at    datetime64[ns, UTC]
summary_kr                   object
raw                          object
created_at           datetime64[ns]
ret_1m                      float64
ret_3m                      float64
ret_10m                     float64
ret_15m                     float64
ret_30m                     float64
ret_60m                     float64
label                        object
score                       float64
dtype: object

결측값 현황:
id                 0
stock_code         0
market             0
company_name       0
report_name        0
disclosed_at       0
su

In [7]:
import sys

sys.path.append("../analysis")

from eda_helper import analyze_disclosure_types_by_keywords, calculate_return_statistics_by_category
ret_columns = ["ret_1m", "ret_3m", "ret_10m", "ret_60m"]
df_with_returns, category_counts = analyze_disclosure_types_by_keywords(target_df, ret_columns = ret_columns)


=== 키워드 기반 공시 유형별 분석 ===
총 40개의 공시 카테고리가 있습니다.

공시 카테고리별 분포:
disclosure_category
실적공시        3722
지분공시        1390
IR활동        1197
공급계약체결      1170
매출변동         424
자사주취득        308
자산양수도        164
투자판단         126
유무상증자        119
전환가액조정       107
조회공시         105
자율공시         103
특허권           90
수시공시          85
자사주처분         74
신규투자          73
소송            60
임상시험계획신청      51
배당            30
전환사채권발행       29
임상시험계획승인      26
자사주소각         25
전환청구권행사       23
장래계획          22
회사합병          22
기술이전          16
품목허가신청        13
자회사경영         13
회사분할          10
영업정지           9
품목허가승인         8
임상시험결과         7
국책과제           7
감자             4
임상시험계획철회       4
영업양수도          2
임상관련           2
해외진출           1
주식교환이전         1
기타             1
Name: count, dtype: int64

수익률 데이터가 있는 공시 건수: 9186
전체 대비 비율: 95.3%

수익률 데이터가 있는 공시 카테고리별 건수:
disclosure_category
실적공시        3614
지분공시        1315
IR활동        1142
공급계약체결      1086
매출변동         378
자사주취득        293
자산양수도        156
투자판단  

In [8]:
import os

# analysis/type 폴더 생성 (없으면 만들기)
type_folder = "../analysis/type"
os.makedirs(type_folder, exist_ok=True)

# 각 카테고리별로 CSV 파일 저장
print("\n=== 카테고리별 CSV 파일 저장 ===")

for category in df_with_returns['disclosure_category'].unique():
    # 해당 카테고리의 데이터 필터링
    category_data = df_with_returns[df_with_returns['disclosure_category'] == category]
    
    # company_name, report_name 컬럼만 선택
    category_data_selected = category_data[['disclosed_at', 'company_name', 'report_name']]
    
    # 파일명 생성 (특수문자 제거)
    safe_filename = category.replace("/", "_").replace("\\", "_").replace(":", "_").replace("*", "_").replace("?", "_").replace("\"", "_").replace("<", "_").replace(">", "_").replace("|", "_")
    filename = f"{safe_filename}.csv"
    filepath = os.path.join(type_folder, filename)
    
    # CSV 파일로 저장
    category_data_selected.to_csv(filepath, index=False, encoding='utf-8-sig')
    
    print(f"{category}: {len(category_data)}건 -> {filename}")

print(f"\n총 {len(df_with_returns['disclosure_category'].unique())}개 카테고리의 CSV 파일이 {type_folder} 폴더에 저장되었습니다.")



=== 카테고리별 CSV 파일 저장 ===
자산양수도: 156건 -> 자산양수도.csv
공급계약체결: 1086건 -> 공급계약체결.csv
신규투자: 71건 -> 신규투자.csv
임상시험계획신청: 50건 -> 임상시험계획신청.csv
자사주처분: 71건 -> 자사주처분.csv
자사주취득: 293건 -> 자사주취득.csv
유무상증자: 116건 -> 유무상증자.csv
IR활동: 1142건 -> IR활동.csv
지분공시: 1315건 -> 지분공시.csv
자율공시: 94건 -> 자율공시.csv
전환가액조정: 94건 -> 전환가액조정.csv
자회사경영: 12건 -> 자회사경영.csv
투자판단: 122건 -> 투자판단.csv
장래계획: 22건 -> 장래계획.csv
실적공시: 3614건 -> 실적공시.csv
조회공시: 100건 -> 조회공시.csv
특허권: 86건 -> 특허권.csv
기술이전: 15건 -> 기술이전.csv
수시공시: 85건 -> 수시공시.csv
임상시험계획철회: 4건 -> 임상시험계획철회.csv
감자: 4건 -> 감자.csv
회사분할: 10건 -> 회사분할.csv
소송: 53건 -> 소송.csv
영업양수도: 2건 -> 영업양수도.csv
임상시험계획승인: 25건 -> 임상시험계획승인.csv
전환사채권발행: 28건 -> 전환사채권발행.csv
임상시험결과: 7건 -> 임상시험결과.csv
매출변동: 378건 -> 매출변동.csv
전환청구권행사: 19건 -> 전환청구권행사.csv
품목허가승인: 8건 -> 품목허가승인.csv
국책과제: 6건 -> 국책과제.csv
회사합병: 22건 -> 회사합병.csv
영업정지: 7건 -> 영업정지.csv
자사주소각: 25건 -> 자사주소각.csv
품목허가신청: 11건 -> 품목허가신청.csv
해외진출: 1건 -> 해외진출.csv
배당: 30건 -> 배당.csv
주식교환이전: 1건 -> 주식교환이전.csv
기타: 1건 -> 기타.csv

총 39개 카테고리의 CSV 파일이 ../analysis/type 폴더에 저장되었습니다.


In [13]:
calculate_return_statistics_by_category(df_with_returns, ret_columns)[["ret_1m_count", "ret_1m_mean", "ret_1m_std", "ret_1m_min", "ret_1m_max", "ret_1m_positive_ratio"]]


=== 공시 카테고리별 수익률 통계 ===

1분 수익률 기준 상위 10개 공시 카테고리:
                     ret_1m_count  ret_1m_mean  ret_1m_std  \
disclosure_category                                          
임상시험결과                          7        1.190       3.391   
감자                              4        0.663       0.869   
소송                             53        0.305       2.099   
품목허가신청                         11        0.202       0.796   
임상시험계획철회                        4        0.135       0.649   
조회공시                          100        0.077       1.239   
유무상증자                         116        0.074       1.220   
공급계약체결                       1086        0.051       1.493   
IR활동                         1142        0.036       0.412   
수시공시                           85        0.035       0.583   

                     ret_1m_positive_ratio  
disclosure_category                         
임상시험결과                                42.9  
감자                                    75.0  
소송                     

,ret_1m_count,ret_1m_mean,ret_1m_std,ret_1m_min,ret_1m_max,ret_1m_positive_ratio
disclosure_category,,,,,,
IR활동,1142,0.036,0.412,-1.59,11.07,29.2
감자,4,0.663,0.869,-0.11,1.85,75.0
공급계약체결,1086,0.051,1.493,-6.09,12.01,39.6
국책과제,6,-0.710,1.467,-3.34,0.59,16.7
기술이전,15,-0.286,0.884,-1.99,1.21,26.7
기타,1,0.000,NaN,0.00,0.00,0.0
매출변동,378,-0.072,0.923,-5.49,6.43,28.6
배당,30,0.004,0.228,-0.78,0.65,20.0
소송,53,0.305,2.099,-1.35,13.90,24.5


In [14]:
calculate_return_statistics_by_category(df_with_returns, ret_columns)[["ret_3m_count", "ret_3m_mean", "ret_3m_std", "ret_3m_min", "ret_3m_max", "ret_3m_positive_ratio"]]


=== 공시 카테고리별 수익률 통계 ===

1분 수익률 기준 상위 10개 공시 카테고리:
                     ret_1m_count  ret_1m_mean  ret_1m_std  \
disclosure_category                                          
임상시험결과                          7        1.190       3.391   
감자                              4        0.663       0.869   
소송                             53        0.305       2.099   
품목허가신청                         11        0.202       0.796   
임상시험계획철회                        4        0.135       0.649   
조회공시                          100        0.077       1.239   
유무상증자                         116        0.074       1.220   
공급계약체결                       1086        0.051       1.493   
IR활동                         1142        0.036       0.412   
수시공시                           85        0.035       0.583   

                     ret_1m_positive_ratio  
disclosure_category                         
임상시험결과                                42.9  
감자                                    75.0  
소송                     

,ret_3m_count,ret_3m_mean,ret_3m_std,ret_3m_min,ret_3m_max,ret_3m_positive_ratio
disclosure_category,,,,,,
IR활동,1130,0.064,0.480,-2.17,8.65,36.5
감자,4,-1.555,1.259,-2.50,0.16,25.0
공급계약체결,1076,-0.139,2.099,-10.63,17.24,39.0
국책과제,6,-1.853,2.499,-6.34,0.24,16.7
기술이전,15,0.705,4.178,-3.44,15.09,40.0
기타,1,-2.020,NaN,-2.02,-2.02,0.0
매출변동,372,-0.165,1.038,-4.83,5.23,34.1
배당,30,0.113,0.267,-0.19,0.94,43.3
소송,53,0.502,2.499,-2.76,13.90,41.5


In [15]:
calculate_return_statistics_by_category(df_with_returns, ret_columns)[["ret_10m_count", "ret_10m_mean", "ret_10m_std", "ret_10m_min", "ret_10m_max", "ret_10m_positive_ratio"]]


=== 공시 카테고리별 수익률 통계 ===

1분 수익률 기준 상위 10개 공시 카테고리:
                     ret_1m_count  ret_1m_mean  ret_1m_std  \
disclosure_category                                          
임상시험결과                          7        1.190       3.391   
감자                              4        0.663       0.869   
소송                             53        0.305       2.099   
품목허가신청                         11        0.202       0.796   
임상시험계획철회                        4        0.135       0.649   
조회공시                          100        0.077       1.239   
유무상증자                         116        0.074       1.220   
공급계약체결                       1086        0.051       1.493   
IR활동                         1142        0.036       0.412   
수시공시                           85        0.035       0.583   

                     ret_1m_positive_ratio  
disclosure_category                         
임상시험결과                                42.9  
감자                                    75.0  
소송                     

,ret_10m_count,ret_10m_mean,ret_10m_std,ret_10m_min,ret_10m_max,ret_10m_positive_ratio
disclosure_category,,,,,,
IR활동,1101,0.105,0.634,-3.41,6.57,45.2
감자,4,-2.322,2.349,-5.37,-0.16,0.0
공급계약체결,1058,-0.538,2.542,-19.29,27.64,32.8
국책과제,5,-2.884,2.394,-5.33,-0.30,0.0
기술이전,15,-0.561,3.331,-5.92,9.17,26.7
기타,1,-3.890,NaN,-3.89,-3.89,0.0
매출변동,358,-0.263,1.659,-13.48,10.86,37.7
배당,29,0.194,0.401,-0.58,1.72,58.6
소송,50,-0.459,1.983,-9.66,4.63,36.0


In [16]:
calculate_return_statistics_by_category(df_with_returns, ret_columns)[["ret_60m_count", "ret_60m_mean", "ret_60m_std", "ret_60m_min", "ret_60m_max", "ret_60m_positive_ratio"]]


=== 공시 카테고리별 수익률 통계 ===

1분 수익률 기준 상위 10개 공시 카테고리:
                     ret_1m_count  ret_1m_mean  ret_1m_std  \
disclosure_category                                          
임상시험결과                          7        1.190       3.391   
감자                              4        0.663       0.869   
소송                             53        0.305       2.099   
품목허가신청                         11        0.202       0.796   
임상시험계획철회                        4        0.135       0.649   
조회공시                          100        0.077       1.239   
유무상증자                         116        0.074       1.220   
공급계약체결                       1086        0.051       1.493   
IR활동                         1142        0.036       0.412   
수시공시                           85        0.035       0.583   

                     ret_1m_positive_ratio  
disclosure_category                         
임상시험결과                                42.9  
감자                                    75.0  
소송                     

,ret_60m_count,ret_60m_mean,ret_60m_std,ret_60m_min,ret_60m_max,ret_60m_positive_ratio
disclosure_category,,,,,,
IR활동,921,0.545,11.774,-5.60,355.34,46.8
감자,4,-2.218,2.122,-4.23,-0.33,0.0
공급계약체결,862,-0.733,3.285,-29.48,27.64,33.5
국책과제,4,-3.128,4.464,-8.26,2.39,25.0
기술이전,12,-1.470,4.984,-14.81,5.88,41.7
기타,1,-6.840,NaN,-6.84,-6.84,0.0
매출변동,257,-0.414,2.145,-13.68,8.64,37.0
배당,27,0.169,0.926,-2.14,2.35,48.1
소송,34,-0.637,1.332,-4.95,0.81,32.4
